In [1]:
import time
import threading

import ipywidgets
import torch
import torchvision
from inputs import get_gamepad
from IPython.display import display
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg
from jetracer.nvidia_racecar import NvidiaRacecar
from torch2trt import TRTModule

In [2]:
#import torch
#import torchvision.transforms as transforms
#import torch.nn.functional as F
#mport cv2
#import numpy as np


#def preprocess(image):
#    device = torch.device('cuda')
#    image = PIL.Image.fromarray(image)
#    image = transforms.functional.to_tensor(image).to(device)
#    image.sub_(mean[:, None, None]).div_(std[:, None, None])
#    return image[None, ...]

In [3]:
SZ = 224

In [4]:
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda() # R, G, B
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

# Read pretrained model

In [5]:
model_trt = TRTModule()
model_trt.load_state_dict(torch.load('models/road_following_model_trt.pth'))

<All keys matched successfully>

# Create RaceCar object

In [6]:
car = NvidiaRacecar()

# Create camera object

In [7]:
camera = CSICamera(width=224, height=224, capture_fps=30) # fps=65

# Driving (AI / manual) thread

In [8]:
preview_widget = ipywidgets.Image(width=SZ, height=SZ)
display(preview_widget)

Image(value=b'', height='224', width='224')

In [9]:
def update_image(change):
    # New image is a numpy array with shape (SZ, SZ, 3), dtype uint8
    image = change['new']
    
    # Convert to tensor and normalize
    t = torchvision.transforms.functional.to_tensor(image).cuda() # TODO: BGR -> RGB!!!
    t.sub_(mean[:, None, None]).div_(std[:, None, None])
    
    # Get center of road
    o = model_trt(t)
    o = torch.clamp(model_trt(t), min=-1, max=1).
    o = (o * 127.5) + 127.5
    o = cpu().list()
    
    # Draw circle and preview
    cv2.circle(image, (x, y), 8, (0, 255, 0), 2)

    jpg = bgr8_to_jpeg(image)
    preview_widget.value = jpg



SyntaxError: invalid syntax (<ipython-input-9-05b6af2bdbc4>, line 11)

In [ ]:
camera.observe(update_image, names='value')

In [ ]:
camera.running = False

In [ ]:
def drive():
    while not done:
        if manual_drive:
            car.throttle = manual_throttle
            car.steering = manual_steering
        else:
            image = camera.read()
            image = preprocess(image).half()
            output = model_trt(image).detach().cpu().numpy().flatten()
            x = float(output[0])
            #print(output)
            car.steering = x #* STEERING_GAIN + STEERING_BIAS
        time.sleep(0)

In [ ]:
image = camera.read()
preview_widget.value = bgr8_to_jpeg(image)



In [ ]:
image[-1,-1]

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(image)

In [ ]:
image.shape

In [ ]:
import PIL

In [ ]:
i1 = PIL.Image.fromarray(image)

In [ ]:
#    image.sub_(mean[:, None, None]).div_(std[:, None, None])



In [ ]:
i1.shape

In [ ]:
i1.dtype

In [ ]:
i1[:,-1,-1]

In [ ]:
i1

In [ ]:
i1 = torch.rand(3)

In [ ]:
i1.tolist()